# Basic ML project

This example shows how to build a Machine Learning pipeline using the Spec API.

Take a look at the `pipeline.yaml` for more details.

## Setup

Make sure you are in the `ml-basic` folder.

```bash .noeval
# setup environment
conda env create --file environment.yml
conda activate ml-basic
```

## Execute the pipeline

In [1]:
%%sh
# make sure output folder exists
mkdir output

# execute using the spec file
ploomber build --entry-point pipeline.yaml

name      Ran?      Elapsed (s)    Percentage
--------  ------  -------------  ------------
get       False               0             0
features  False               0             0
join      False               0             0
fit.py    False               0             0


mkdir: output: File exists
Rendering DAG: 100%|██████████| 4/4 [00:00<00:00, 13.15it/s]
4it [00:00, 23399.19it/s]


Note: `ploomber build` also works because Ploomber looks for a `pipeline.yaml` file by default.